In [13]:
import sys
sys.path.append('../')

import jraph
import jax.numpy as jnp
import jax.random as jr
import jax

import equiformer.graphs as graphs
import equiformer.layers as layers

from jax.lax import gather

In [21]:
n_channels = {0: 1, 1: 1}
graph = graphs.create_rand_graph(100, 200, n_channels, jr.PRNGKey(0))

In [22]:
tp_layer = layers.TensorProductLayer({0: [0, 1], 1: [0, 1]}, n_channels, l_max=1)

In [23]:
@jax.jit
def apply(graph):
    return tp_layer(graph)


graph_out = apply(graph)

In [24]:
%timeit apply(graph).nodes[1].block_until_ready()

33 µs ± 301 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [25]:
%timeit tp_layer(graph).nodes[1].block_until_ready()

9.4 ms ± 721 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
graph_out

GraphsTuple(nodes={-1: Array([[-0.38345885,  0.6296871 , -0.15880105],
       [ 0.25808525,  0.49625078, -0.2821705 ],
       [ 0.19775973,  0.00491796,  0.23706111],
       [ 0.06808257, -0.31465647, -1.2616131 ],
       [-0.42206448, -0.08140673, -0.5123282 ],
       [-0.25323692,  1.104096  ,  1.8927003 ],
       [ 1.9079154 ,  1.2675216 , -0.16225533],
       [ 0.92891777, -0.00534184,  0.9454967 ],
       [-0.19104688, -0.57152027, -0.5819773 ],
       [ 0.8557095 , -0.5239636 , -1.8662931 ]], dtype=float32), 0: Array([[[ 0.01028168],
        [ 0.01339297]],

       [[ 1.3520383 ],
        [ 1.0296303 ]],

       [[ 1.5869076 ],
        [        nan]],

       [[ 0.48660356],
        [-0.57795244]],

       [[ 0.        ],
        [ 0.        ]],

       [[ 0.7106478 ],
        [ 0.58071756]],

       [[ 0.24086651],
        [ 0.53447354]],

       [[ 0.18795948],
        [ 0.13502236]],

       [[ 0.23992814],
        [-0.39924896]],

       [[ 0.4768452 ],
        [-0.44780803]]

In [11]:
graph_out.nodes[0].shape

(10, 2, 1)

In [10]:
graph_out.edges.shape

(20, 3)